In [1]:
library(here)
library(here)
library(readr, warn.conflicts = FALSE)
library(arrow, warn.conflicts = FALSE)
library(magrittr, warn.conflicts = FALSE)
library(stringr, warn.conflicts = FALSE)
library(dplyr, warn.conflicts = FALSE)
library(rlang, warn.conflicts = FALSE)
library(data.table, warn.conflicts = FALSE)
library(lubridate, warn.conflicts = FALSE)
library(tidyr, warn.conflicts = FALSE)
library(truveta.study)
library(dbplyr)
library(lubridate)

here() starts at /home/jovyan/studyrepositories/hypertensioncohorts-is5fyzbqztbebenalfbuo4boem_backup

Loading required package: DBI


Attaching package: ‘dbplyr’


The following objects are masked from ‘package:dplyr’:

    ident, sql




In [2]:
# build connections 
con <- create_connection()
study <- get_study(con, title = "Hypertension Cohort")
population <- get_population(con, study, title = "Study Cohort")
snapshot <- get_latest_snapshot(con, population, finished = TRUE)

[1] "Deprecation warning: the Vital and SocialHistory tables will be removed from the Truveta Data Model on September 12th, 2023. New snapshots generated after this date will exclude these tables. Existing snapshots which include these tables will be unaffected. Visit Learn for more information."


To sign in to Truveta, use a web browser to open the page https://login.truveta.com/activate?user_code=WKHH-WKWP.

Signed in successfully.



In [6]:
#basic encounter table counts

select <- "select count(Id) as totalrow, count(distinct PersonId) as patient
from Encounter;"

load_sql_table(con,snapshot,query = select)%>% collect()


sql <-  "select count(Id) as totalrow,count(distinct PersonId) as patient
from Encounter
where StartDateTime is not NULL;"


load_sql_table(con,snapshot,query = sql)%>% collect()

sql_filter <- " select count(Id) as totalrow,count(distinct PersonId) as patient
from Encounter
where ClassConceptId !=1067555
 and StatusConceptId not in (1067555,2983200,2506595, 2983199,2506590)
"
load_sql_table(con,snapshot,query = sql_filter)%>% collect()

sql2 <- "select count(Id) as total,count(distinct PersonId) as patient
from Encounter 
where Id in
(select distinct EncounterId
from Condition);"

load_sql_table(con,snapshot,query = sql2)%>% collect()

sql3 <- "select count(Id) as total,count(distinct PersonId) as patient
from Encounter 
where Id not in
(select distinct EncounterId
from Condition);"


load_sql_table(con,snapshot,query = sql3)%>% collect()



totalrow,patient
<int>,<int>
255000306,1220874


totalrow,patient
<int>,<int>
254438088,1220769


totalrow,patient
<int>,<int>
237822156,1220673


total,patient
<int>,<int>
110026313,1219568


total,patient
<int>,<int>
0,0


In [14]:
get_data_dictionary(con,snapshot)%>% arrange(table) %>% filter(table =="Procedure")

table,column,column_type
<chr>,<chr>,<chr>
Procedure,RecordedDateTime,datetime2
Procedure,StartDateTime,datetime2
Procedure,StatusConceptId,int
Procedure,PersonId,varchar
Procedure,SourceProvenanceConceptId,int
Procedure,CategoryConceptId,int
Procedure,EncounterId,varchar
Procedure,EndDateTime,datetime2
Procedure,Id,varchar


In [3]:
# encounter in procedure

procedure <- "select count(Id) as nrow, count(distinct PersonId) as patient
from Encounter 
where Id in
(select distinct EncounterId 
from [Procedure])
"

load_sql_table(con,snapshot,query = procedure)%>% collect()


notprocedure <- "select count(Id) as nrow, count(distinct PersonId) as patient
from Encounter 
where Id not in
(select distinct EncounterId
from [Procedure])"

load_sql_table(con,snapshot,query = notprocedure)%>% collect()

nrow,patient
<int>,<int>
27176397,837208


nrow,patient
<int>,<int>
0,0


In [23]:
either <- "select count(distinct c.EncounterId) as crecords, 
count(distinct p.EncounterId) as precords,
count(distinct c.PersonId) as conpatient,
count(distinct p.PersonId) as ppatient
from 
Condition c full outer join [Procedure] p on (c.EncounterId = p.EncounterId);"

either_tb <- load_sql_table(con,snapshot,query = either)%>% collect() 



both <- "select count(distinct c.EncounterId) as records,count(distinct c.PersonId) as patient 
from Condition c inner join [Procedure] p on (c.EncounterId = p.EncounterId);"

both_tb <- load_sql_table(con,snapshot,query =both)%>% collect()

total <- either_tb$crecords + either_tb$precords - both_tb$records
total

crecords,precords,conpatient,ppatient
<int>,<int>,<int>,<int>
110030147,27177100,1221009,837236


records,patient
<int>,<int>
21243369,822869


In [14]:
# get encounter as patient * start time

sql <-"select count(*) as total
from
(select e.PersonId, e.StartDateTime, count(distinct e.Id) as rowtotal
from Encounter e 
group by PersonId, StartDateTime ) as t
"

new_enct <- load_sql_table(con,snapshot,query=sql)%>% collect()
new_enct

# get condition encounter as patient * start time

sql <- "select count(*) as total
from
(select c.PersonId, e.StartDateTime, count(distinct c.EncounterId) as rowtotal
from Encounter e inner join Condition c on (e.Id = c.EncounterId)
group by c.PersonId, e.StartDateTime ) as t
"

new_con_enct <- load_sql_table(con,snapshot,query=sql)%>% collect()
new_con_enct

#get procedure encounter as patient * start time

sql <- "select count(*) as total
from
(select c.PersonId, e.StartDateTime, count(distinct c.EncounterId) as rowtotal
from Encounter e inner join [Procedure] c on (e.Id = c.EncounterId)
group by c.PersonId, e.StartDateTime ) as t
"

new_pro_enct <- load_sql_table(con,snapshot,query=sql)%>% collect()
new_pro_enct

total
<int>
195215705


total
<int>
97722533


total
<int>
26564431


In [5]:
#basic encounter table counts

select <- "select count(Id) as totalrow
from Encounter;"

load_sql_table(con,snapshot,query = select)%>% collect()


sql <-  "select count(Id) as totalrow
from Encounter
where StartDateTime is not NULL;"

load_sql_table(con,snapshot,query = sql)%>% collect()

sql2 <- "select count(Id) as total
from Encounter 
where Id in
(select distinct EncounterId
from Condition);"

load_sql_table(con,snapshot,query = sql2)%>% collect()

sql3 <- "select count(Id) as total
from Encounter 
where Id not in
(select distinct EncounterId
from Condition);"


load_sql_table(con,snapshot,query = sql3)%>% collect()



totalrow
<int>
255000306


totalrow
<int>
254438088


total
<int>
110026313


total
<int>
0


In [5]:
# basic summary for variables in encounter table

sql <- "select e.ClassConceptId, c.ConceptName, count(*) as total
from Encounter e join Concept c on (e.ClassConceptId = c.ConceptId)
group by ClassConceptId, c.ConceptName
order by total;"

load_sql_table(con,snapshot,query = sql) %>% collect()

sql <- "select e.DischargeDispositionConceptId, c.ConceptName, count(*) as total
from Encounter e join Concept c on (e.DischargeDispositionConceptId = c.ConceptId)
group by DischargeDispositionConceptId, c.ConceptName
order by total;"

load_sql_table(con,snapshot,query = sql) %>% collect()

sql <- "select e.PriorityConceptId, c.ConceptName, count(*) as total
from Encounter e join Concept c on (e.PriorityConceptId = c.ConceptId)
group by e.PriorityConceptId, c.ConceptName
order by total;"

load_sql_table(con,snapshot,query = sql) %>% collect()


sql <- "select e.StatusConceptId, c.ConceptName, count(*) as total
from Encounter e join Concept c on (e.StatusConceptId = c.ConceptId)
group by e.StatusConceptId, c.ConceptName
order by total"

load_sql_table(con,snapshot,query = sql) %>% collect()



sql <- "select e.TypeConceptId, c.ConceptName, count(*) as total
from Encounter e join Concept c on (e.TypeConceptId = c.ConceptId)
group by e.TypeConceptId, c.ConceptName
order by total"

load_sql_table(con,snapshot,query = sql) %>% collect()

ClassConceptId,ConceptName,total
<int>,<chr>,<int>
1065218,field,297
1065215,inpatient acute,5234
1067555,Invalid,6975
1065226,pre-admission,42375
1067558,Unknown,217324
1065225,observation encounter,617645
1065227,short stay,1001276
0,Field has not been mapped,2336671
1065219,home health,4571188


DischargeDispositionConceptId,ConceptName,total
<int>,<chr>,<int>
1065238,Federal hospital,532
1067555,Invalid,721
1065234,Cancer research/childrens hospital,1575
2649591,Insufficient Information,1719
1065232,Re-admitted,2712
1065241,Court/law enforcement,4196
1065236,Custodial/Suportive faciilty,5246
1065242,Long-term care,6622
1065235,Critical accesss hospital,8609


PriorityConceptId,ConceptName,total
<int>,<chr>,<int>
1065253,Emergency Index Severity Level 1,10225
2506580,emergency,17752
2506588,urgent,206899
1065256,Emergency Index Severity Level 4,236451
1065255,Emergency Index Severity Level 3,447518
0,Field has not been mapped,474405
1065254,Emergency Index Severity Level 2,509767
1065257,Emergency Index Severity Level 5,1353752
1067557,Field is not present in source,76114466


StatusConceptId,ConceptName,total
<int>,<chr>,<int>
1067555,Invalid,2
1065264,Patient Transfer,32
2506593,In Progress,5869
0,Field has not been mapped,18758
2983200,Left without being seen,30021
1065266,Pre Admission,59885
1065261,Patient Admission,76772
2506597,Unknown,164501
2506589,Arrived,441108


TypeConceptId,ConceptName,total
<int>,<chr>,<int>
1065315,Nuclear medicine,1
1065314,Newborn,1
1065322,Organ Procurement,8
1065302,Inpatient Preadmit,8
1066382,Palliative medicine,9
3059275,Swing,9
1065285,Diagnostic Extended,9
3059268,Baby,11
1065300,Inpatient Hospice,13


In [3]:
# diagnosis encounter - condition summary

concept = read_parquet("data/Concept.parquet")

sql <- "select c.CategoryConceptId, count(distinct c.Id) as total 
from Encounter e left join Condition c on (e.Id = c.EncounterId)
group by CategoryConceptId
order by total desc;"

cat <- load_sql_table(con,snapshot,query = sql) %>% collect %>% merge(., concept,by.x = "CategoryConceptId", by.y = "ConceptId")
cat

sql <- "select c.PrimaryDiagnosisConceptId, count(distinct c.Id) as cnt from
Encounter e left join Condition c on (e.Id = c.EncounterId)
 group by PrimaryDiagnosisConceptId
 order by PrimaryDiagnosisConceptId;"
primdx <- load_sql_table(con, snapshot, query=sql) %>% collect()
primdx <- primdx %>% left_join(concept, by = c('PrimaryDiagnosisConceptId' = 'ConceptId'))
primdx

sql <- "select c.ClinicalStatusConceptId, count(distinct c.Id) as cnt from
 Encounter e left join Condition c on (e.Id = c.EncounterId)
 group by ClinicalStatusConceptId
 order by ClinicalStatusConceptId;"
clinstat <- load_sql_table(con, snapshot, query=sql) %>% collect()
clinstat <- clinstat %>% left_join(concept, by = c('ClinicalStatusConceptId' = 'ConceptId'))
clinstat

sql <- "select c.SeverityConceptId, count(distinct c.Id) as cnt from
 Encounter e left join Condition c on (e.Id = c.EncounterId)
 where RecordedDateTime is not null
 group by SeverityConceptId
 order by SeverityConceptId;"
sevstat <- load_sql_table(con, snapshot, query=sql) %>% collect()
sevstat <- sevstat %>% left_join(concept, by = c('SeverityConceptId' = 'ConceptId'))
sevstat


sql <- "select c.VerificationStatusConceptId, count(distinct c.Id) as cnt from 
 Encounter e left join Condition c on (e.Id = c.EncounterId)
 group by VerificationStatusConceptId
 order by VerificationStatusConceptId;"
veristat <- load_sql_table(con, snapshot, query=sql) %>% collect()
veristat <- veristat %>% left_join(concept, by = c('VerificationStatusConceptId' = 'ConceptId'))
veristat

sql <- "select c.SourceProvenanceConceptId, count(distinct c.Id) as cnt from
 Encounter e left join Condition c on (e.Id = c.EncounterId)
 where RecordedDateTime is not null
 group by SourceProvenanceConceptId
 order by SourceProvenanceConceptId;"
srcprov <- load_sql_table(con, snapshot, query=sql) %>% collect()
srcprov <- srcprov %>% left_join(concept, by = c('SourceProvenanceConceptId' = 'ConceptId'))
srcprov


CategoryConceptId,total,ConceptName,ConceptDefinition,CodeSystem,ConceptCode,ConceptClass,Domain,__index_level_0__
<dbl>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>
0,55285,Field has not been mapped,The field has not been mapped.,NullFlavor,NM,NullFlavor,Categories,22724
1065168,33045861,AdmissionDiagnosis,AdmissionDiagnosis,Truveta,1065168,ConditionCategory,Categories,264675
1065169,140627640,BillingDiagnosis,BillingDiagnosis,Truveta,1065169,ConditionCategory,Categories,264676
1065170,209327512,EncounterDiagnosis,EncounterDiagnosis,Truveta,1065170,ConditionCategory,Categories,264677
1065172,251512834,MedicalHistory,MedicalHistory,Truveta,1065172,ConditionCategory,Categories,264679
1065174,4247590,ProblemList,ProblemList,Truveta,1065174,ConditionCategory,Categories,264680
1067561,427457,No Information,"The value is exceptional (missing, omitted, incomplete, improper). No information as to the reason for being an exceptional value is provided. This is the most general exceptional value. It is also the default exceptional value.",NullFlavor,NI,NullFlavor,Categories,266949
2649591,296522,Insufficient Information,Insufficient Information,NullFlavor,II,NullFlavor,Categories,81742


PrimaryDiagnosisConceptId,cnt,ConceptName,ConceptDefinition,CodeSystem,ConceptCode,ConceptClass,Domain,__index_level_0__
<dbl>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>
NA,0,NA,NA,NA,NA,NA,NA,NA
1067557,234069050,Field is not present in source,Field is not present in source,NullFlavor,FNPS,NullFlavor,Categories,266946
1067561,1792009,No Information,"The value is exceptional (missing, omitted, incomplete, improper). No information as to the reason for being an exceptional value is provided. This is the most general exceptional value. It is also the default exceptional value.",NullFlavor,NI,NullFlavor,Categories,266949
1200405,342664693,No,No,Truveta,1200405,YesNoIndicator,Categories,90723
1200406,61014949,Yes,Yes,Truveta,1200406,YesNoIndicator,Categories,90724


ClinicalStatusConceptId,cnt,ConceptName,ConceptDefinition,CodeSystem,ConceptCode,ConceptClass,Domain,__index_level_0__
<dbl>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>
NA,0,NA,NA,NA,NA,NA,NA,NA
1065181,1,Recurrence,"The subject is experiencing a re-occurence or repeating of a previously resolved condition, e.g. urinary tract infection, pancreatitis, cholangitis, conjunctivitis.",Truveta,1065181,ConditionClinicalStatus,Categories,264684
1067557,552749391,Field is not present in source,Field is not present in source,NullFlavor,FNPS,NullFlavor,Categories,266946
1067561,86791309,No Information,"The value is exceptional (missing, omitted, incomplete, improper). No information as to the reason for being an exceptional value is provided. This is the most general exceptional value. It is also the default exceptional value.",NullFlavor,NI,NullFlavor,Categories,266949


SeverityConceptId,cnt,ConceptName,ConceptDefinition,CodeSystem,ConceptCode,ConceptClass,Domain,__index_level_0__
<int>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>
1065187,342791,Severe,Severe,Truveta,1065187,ConditionSeverity,Categories,264688
1065188,4084264,Mild,Mild,Truveta,1065188,ConditionSeverity,Categories,264689
1065189,959662,Moderate,Moderate,Truveta,1065189,ConditionSeverity,Categories,264690
1067555,406430,Invalid,The value as represented in the instance is not a member of the set of permitted data values in the constrained value domain of a variable.,NullFlavor,INV,NullFlavor,Categories,266944
1067557,436736074,Field is not present in source,Field is not present in source,NullFlavor,FNPS,NullFlavor,Categories,266946
1067558,112263,Unknown,"A proper value is applicable, but not known.",NullFlavor,UNK,NullFlavor,Categories,266947
1067561,71842120,No Information,"The value is exceptional (missing, omitted, incomplete, improper). No information as to the reason for being an exceptional value is provided. This is the most general exceptional value. It is also the default exceptional value.",NullFlavor,NI,NullFlavor,Categories,266949
2649591,2,Insufficient Information,Insufficient Information,NullFlavor,II,NullFlavor,Categories,81742


VerificationStatusConceptId,cnt,ConceptName,ConceptDefinition,CodeSystem,ConceptCode,ConceptClass,Domain,__index_level_0__
<dbl>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>
NA,0,NA,NA,NA,NA,NA,NA,NA
0,4030592,Field has not been mapped,The field has not been mapped.,NullFlavor,NM,NullFlavor,Categories,22724
1065193,337334,Confirmed,There is sufficient diagnostic and/or clinical evidence to treat this as a confirmed condition.,Truveta,1065193,ConditionVerificationStatus,Categories,264691
1065194,10,Differential,One of a set of potential (and typically mutually exclusive) diagnoses asserted to further guide the diagnostic process and preliminary treatment.,Truveta,1065194,ConditionVerificationStatus,Categories,264692
1065197,6307,Provisional,This is a tentative diagnosis - still a candidate that is under consideration.,Truveta,1065197,ConditionVerificationStatus,Categories,264694
1065198,27,Refuted,This condition has been ruled out by diagnostic and clinical evidence.,Truveta,1065198,ConditionVerificationStatus,Categories,264695
1065199,14,Unconfirmed,There is not sufficient diagnostic and/or clinical evidence to treat this as a confirmed condition.,Truveta,1065199,ConditionVerificationStatus,Categories,264696
1067557,543569781,Field is not present in source,Field is not present in source,NullFlavor,FNPS,NullFlavor,Categories,266946
1067561,91596636,No Information,"The value is exceptional (missing, omitted, incomplete, improper). No information as to the reason for being an exceptional value is provided. This is the most general exceptional value. It is also the default exceptional value.",NullFlavor,NI,NullFlavor,Categories,266949


SourceProvenanceConceptId,cnt,ConceptName,ConceptDefinition,CodeSystem,ConceptCode,ConceptClass,Domain,__index_level_0__
<dbl>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>
NA,645249,NA,NA,NA,NA,NA,NA,NA
0,513838008,Field has not been mapped,The field has not been mapped.,NullFlavor,NM,NullFlavor,Categories,22724
3056721,349,Notes,Notes,Truveta,3056721,SourceProvenance,Categories,269835


In [5]:
sum(cat$total)

[1] 130042371

In [8]:
concept_table <- load_sql_table(con,snapshot, query = "select * from Concept;")%>% collect()
write_csv(concept_table,"data/ConceptTable.csv.r")

In [29]:
# diagnosis condition encounter filtered

sql <- "select count(distinct e.Id) as totalrow,count(distinct e.PersonId) as patient
from Encounter e inner join Condition c on (e.Id = c.EncounterId)
where ClassConceptId !=1067555 and StartDateTime is not NULL
 and StatusConceptId not in (1067555,2983200,2506595, 2983199,2506590) "

diag_enct <- load_sql_table(con,snapshot,query = sql)%>% collect()
diag_enct

totalrow,patient
<int>,<int>
107231531,1215632


In [30]:
# diagnosis procedure encounter filtered

sql <- "select count(distinct e.Id) as totalrow,count(distinct e.PersonId) as patient
from Encounter e inner join [Procedure] c on (e.Id = c.EncounterId)
where e.ClassConceptId !=1067555 and e.StartDateTime is not NULL
 and e.StatusConceptId not in (1067555,2983200,2506595, 2983199,2506590) "

diag_enct <- load_sql_table(con,snapshot,query = sql)%>% collect()
diag_enct

totalrow,patient
<int>,<int>
26083400,823870


In [ ]:
# diagnosis encounter filtered

sql <- "select e.ClassConceptId, count(c.Id) as totalrow,count(distinct c.PersonId) as patient
from Encounter e left join Procedure c on (e.Id = c.EncounterId)
where ClassConceptId !=1067555 and StartDateTime is not NULL
 and StatusConceptId not in (1067555,2983200,2506595, 2983199,2506590) "

proce_enct <- load_sql_table(con,snapshot,query = sql)%>% collect()

In [ ]:
# diagnosis encounter - condition summary

concept <- read_parquet("data/Concept.parquet")

sql <- "select e.ClassConceptId, count(distinct c.EncounterId) as total ,count(distinct c.PersonId) as patients
from Encounter e inner join Condition c on (e.Id = c.EncounterId)
group by ClassConceptId
order by total desc;"

cla <- load_sql_table(con,snapshot,query = sql) %>% collect %>% merge(., concept,by.x = "ClassConceptId", by.y = "ConceptId")
cla

sql <- "select e.StatusConceptId, count(distinct c.EncounterId) as total, count(distinct c.PersonId) as patients from
Encounter e inner join Condition c on (e.Id = c.EncounterId)
 group by e.StatusConceptId
 order by total desc;"
stat <- load_sql_table(con, snapshot, query=sql) %>% collect()%>% left_join(concept, by = c('StatusConceptId' = 'ConceptId'))
stat

sql <- "select e.TypeConceptId, count(distinct c.EncounterId) as total, count(distinct c.PersonId) as patients from
 Encounter e inner join Condition c on (e.Id = c.EncounterId)
 group by e.TypeConceptId
 order by total desc;" 
typ <- load_sql_table(con, snapshot, query=sql) %>% collect() %>% left_join(concept, by = c('TypeConceptId' = 'ConceptId'))
typ


In [ ]:
# diagnosis encounter - Procedure summary

concept <- read_parquet("data/Concept.parquet")

sql <- "select e.ClassConceptId, count(distinct c.EncounterId) as total ,count(distinct c.PersonId) as patients
from Encounter e inner join [Procedure] c on (e.Id = c.EncounterId)
group by e.ClassConceptId
order by total desc;"

clap <- load_sql_table(con,snapshot,query = sql) %>% collect %>% merge(., concept,by.x = "ClassConceptId", by.y = "ConceptId")
clap

sql <- "select e.StatusConceptId, count(distinct c.EncounterId) as total, count(distinct c.PersonId) as patients from
Encounter e inner join [Procedure] c on (e.Id = c.EncounterId)
 group by e.StatusConceptId
 order by total desc;"
statp <- load_sql_table(con, snapshot, query=sql) %>% collect()%>% left_join(concept, by = c('StatusConceptId' = 'ConceptId'))
statp

sql <- "select e.TypeConceptId, count(distinct c.EncounterId) as total, count(distinct c.PersonId) as patients from
 Encounter e inner join [Procedure] c on (e.Id = c.EncounterId)
 group by e.TypeConceptId
 order by total desc;" 
typp <- load_sql_table(con, snapshot, query=sql) %>% collect() %>% left_join(concept, by = c('TypeConceptId' = 'ConceptId'))
typp


In [ ]:
# types <- merge(typ,typp, by.x = "TypeConceptId",by.y = "TypeConceptId")%>% 

In [ ]:
write_csv(typ, "data/de_typeclass_condition.csv.r")
write_csv(typp, "data/de_typeclass_procedure.csv.r")

In [3]:
# get defined diagnosis encounter: condition + procedure

diag_ent <- "
select distinct EncounterId from
(select distinct EncounterId from Condition
union
select distinct EncounterId from [Procedure]) as t
"

diag_encounter<- load_sql_table(con,snapshot,query = diag_ent)%>% collect()
summary(diag_encounter)

 EncounterId       
 Length:115963879  
 Class :character  
 Mode  :character  

In [4]:
summary(diag_encounter)
write_parquet(diag_encounter, "data/diag_enct_id.parquet")

 EncounterId       
 Length:115963879  
 Class :character  
 Mode  :character  

In [6]:
# non-diagnosis encounter - summary
diag_encounter <- read_parquet("data/diag_enct_id.parquet")
concept <- read_parquet("data/Concept.parquet")
params <- list('encounterid' = diag_encounter)

non_diag <- "select e.ClassConceptId,count(distinct e.Id) as Records, count(distinct e.PersonId) as Patients
from Encounter e 
where e.Id not in :encounterid
group by e.ClassConceptId
order by Records desc"

non_diag_class<- load_sql_table(con,snapshot,query = non_diag,params = params)%>% collect() %>% merge(.,concept,by.x = "ClassConceptId",
                                                                                                      by.y = "ConceptId")
non_diag_class



In [7]:
non_diag_class

ERROR: Error in eval(expr, envir, enclos): object 'non_diag_class' not found


In [9]:

non_diag <- "select e.TypeConceptId, c.ConceptName, count(distinct e.Id) as Records, count(distinct e.PersonId) as Patients
from Encounter e left join Concept c on(e.ClassConceptId = c.ConceptId)
where e.Id not in :encounterid
group by e.TypeConceptId, ConceptName
order by Records desc"

non_diag_type<- load_sql_table(con,snapshot,query = non_diag,params = params)%>% collect()

non_diag_type

non_diag <- "select e.StatusConceptId, c.ConceptName, count(distinct e.Id) as Records, count(distinct e.PersonId) as Patients
from Encounter e left join Concept c on(e.ClassConceptId = c.ConceptId)
where e.Id not in :encounterid
group by e.StatusConceptId, ConceptName
order by Records desc"

non_diag_stat<- load_sql_table(con,snapshot,query = non_diag,params = params)%>% collect()

non_diag_stat

ClassConceptId,ConceptName,Records,Patients
<???>,<???>,<???>,<???>
